# Assignment 2

We start by giving a simple overview of the implementation. Then we will show the necessary vode for applying the NN-class made and the resulting results.

## Performance measure
The model performance is calculated by the accuracy of the estimated model when applied to the validation data. The accuracy is the number of correct predictions divided by the number of total observations. <br>

The model output is not on binary form. In order to calculate the accuracy, we transform the model output to binary form by use of the Hard max function. The Hard max function sets the largest element value in the putput vectors equal to one and the other elements to zero. 

## The main algorithm
We will apply the method "solAlg1" from the Neural Network class "nnClass.py". The algorithm goes forward and backward for each observation. Before every epoch the order of the training observations is shuffled. 

## Early stopping
The main algorithm "solAlg1" contains early stopping. The algorithm will run for a chosen number of epochs before the model performance on the validation set is calculated. When the solution has not improved in a user given number of periods, this is a sign of the network having found the global optimum, and the simulations end. We allow the validation set performance to drop for several runs in order to less easily get trapped in local optima.

## The main algorithm pseudo code
Here follows pseudo code of the main algorithm:

solAlg1:
        
        while localOptima < maxLocalOptima and number of validations < maxValidations:
            for trainingCycle in range(trainingCyclesPerValidation):
                shuffle index
                for idx in range(np.shape(self.targetMatrixTrain)[0]):
                    forward
                    backward
            calculate validation set performance
            
            if performance lower than best performance
                localOptima += 1
            else:
                store new performance score
            validationIdx += 1

## Code
The code is found in two files: <br>
1) nnClass.py: contains the Neural network solver class. <br>
2) runAssignment2.py:  contains a function for loading of the assignment data and functions for solving the specific tasks. <br>

We start by importing everything from the above mentioned  files.

In [1]:
from nnClass import *
from runAssignment2 import *

"dataAssignment2", which is part of spyerTesting, split the data into training/velidation/test with the corresponding relations 50/25/25 per cent. We will first use this data split when analysing the model performance. By calling the function "dataAssignment2", we load the data for the assignment and split it into test, validation and training sets. "dataAssignment2" is almost identical to the file "movements.py" provided with the assignment.

In [2]:
train, train_targets, valid, valid_targets, test, test_targets = dataAssignment2()

# Hidden nodes. Fixed validation set.
In order to determine the optimal number of hidden nodes, we run the neural network with different number of nodes and study the model performance on validation sets. The solution algorith above is applied. In addition to the main part of the solution algorithm, the algorithm produces confusion matrices and accuracy scores. The implementation and results for different number of hidden nodes now follow. 

In [3]:
activationFunction= 'sigmoid'
trainingCyclesPerValidation = 5
maxValidations= 300
maxLocalOptima = 15

for numberOfHiddenNodes in 6, 8 ,12:
    fixedValidationSet( train=train,
                       train_targets = train_targets,
                       valid=valid,
                       valid_targets=valid_targets,
                       numberOfHiddenNodes = numberOfHiddenNodes,
                        activationFunction = activationFunction,
                        trainingCyclesPerValidation = trainingCyclesPerValidation,
                        maxValidations = maxValidations,
                        maxLocalOptima = maxLocalOptima)


Hidden nodes:  6
Activation function:  sigmoid
Hidden nodes:  6
Epochs per validation:  5
Best Confusion Matrix: 
 [[0.67 0.   0.   0.   0.33 0.   0.   0.  ]
 [0.   1.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.26 0.   0.26 0.37 0.   0.   0.11]
 [0.   0.   0.   0.   1.   0.   0.   0.  ]
 [0.   0.07 0.   0.   0.14 0.71 0.07 0.  ]
 [0.   0.   0.   0.   0.   0.   1.   0.  ]
 [0.06 0.   0.06 0.   0.   0.   0.   0.89]]
Best accuracy: 0.82
TotalNumberOfIterations:  160
Validations:  32

Hidden nodes:  8
Activation function:  sigmoid
Hidden nodes:  8
Epochs per validation:  5
Best Confusion Matrix: 
 [[0.94 0.   0.06 0.   0.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.79 0.   0.   0.   0.21]
 [0.   0.   0.   0.18 0.82 0.   0.   0.  ]
 [0.   0.07 0.   0.   0.   0.93 0.   0.  ]
 [0.   0.   0.07 0.   0.   0.   0.93 0.  ]
 [0.   0.   0.   0.17 0.   0.   0.   0.83]]
Best accura

__Warning:__ The results change with new runs due to the randomness in the main algorithm. Hence re-running the above code will give different results, especially wrt. the confusion matrix and which classes are worst.<br>

We see that the accuracy lays between 85 and 93 percent, depending on the number of hidden nodes. <br>

There is not a linear relation between number of nodes and model performance. The middle node number does best. However, we must take into account the randomness of the algorithm. The training observations are shuffled between each epcoh. Hence the performance will never stay the same for one solution, and the above results are just a suggestion. In order to study the optimal node number further, we could have solved for each node number many times and used average performance. <br>

The number of simulations seems to be about the same irrespective of node numbers. This is a bit surprising, since I would expect a more complex network needing more simulations. <br>

Based on the above, which is uncertain, it looks like 8 nodes is the best of the provided alternatives. There are five classes that has mis-classifications in the case of 8 nodes. The classes with mis-classifications are classes 3-7. Clas 6 is worst, since it has the lowest share of correct classifications. <br>

The mis-classfication pattern differst between the different node numbers.

# K-fold
Next we will apply the K-fold crossvalidation method, which is a resempling method. All the data excluding the test data is divided into $K$ number of folds. For one iteration one of the folds performs as validation set while the remaining folds perform as training set. <br>

K-fold is implemented as an own method in the NN-class. The traning and validation sets are concatenated to one data set. The resulting data set is then divided into $K$ folds. Each combination of folds are sent into the main solution algorithm descirbed and applied above. Hence we will get $K$ number of results from the $K$-fold analysis.<br>

In order to compare the K-fold method with the fixed validation set method, we choose the same number of epochs per validation and the same limit for worsening solutions. Furthermore we choose $K=3$ so that the size of the training and validation sets mimimcs the size from the previous analysis with fixed training and validation sets.

In [8]:
for numberOfHiddenNodes in 6, 8 ,12:
    print('\n#####################   ', 'Hidden nodes: ', numberOfHiddenNodes, '   ############################')
    runKfold(train=train ,
            train_targets=train_targets,
            valid=valid ,
            valid_targets=valid_targets ,
            numberOfHiddenNodes=numberOfHiddenNodes,
            activationFunction= 'sigmoid',
            numberOfFolds = 3, 
            trainingCyclesPerValidation=5,
            maxValidations = 1000,
            maxLocalOptima = 15,
            printConfusionMatrix=False)


#####################    Hidden nodes:  6    ############################

Best accuracy folds:  [0.8700000000000001, 0.8623279098873592, 0.7825]
Mean(bestScores): 0.84, std(bestScores) 0.04

#####################    Hidden nodes:  8    ############################

Best accuracy folds:  [0.9199999999999999, 0.8826466916354557, 0.8377028714107365]
Mean(bestScores): 0.88, std(bestScores) 0.03

#####################    Hidden nodes:  12    ############################

Best accuracy folds:  [0.8998748435544429, 0.8987499999999999, 0.9025000000000001]
Mean(bestScores): 0.90, std(bestScores) 0.00


We see that the accuracy scores lay in the same area as for the static train/test split, the mean scores being between approximately 80 and 90 per cent. The rankings between the different hidden node numbers differs somewhat from the static train/test split: the highest node number is still the best, but the lowest node number is now the worst. <br>

The standard deviations gives extra information about the uncertainty of the different scores. We see that the standard deviations are so large for the two lowest node numebrs that the typical 95 per cent confidence intervals for normally distributed variables for the two lowest nodes overlaps. This also suggests that the rankings of the methods with the static method should be based on repetition of the simulations, as suggested earlier. <br>

The highest node number case has almost no uncertainty. However, I other runs have given larger standard deviations also for this node number. <br>

We conclude that the network performs about the same independent on whether static splitting or K-fold cross validation is used. Furthermore, the accuracy differences seem to be very uncertain, suggesting that the difference in quality between the chosen node numbers might not be that big.